In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path("..").resolve()))

In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon

In [ ]:
from fractopo import branches_and_nodes, contour_grid
from fractopo.tval import executor
from fractopo.analysis import main

In [ ]:
trace_data = ["../tests/sample_data/KB11_traces.shp"]
area_data = ["../tests/sample_data/KB11_area.shp"]

assert all([Path(filepath).exists() for filepath in trace_data+area_data])

In [ ]:
traces = [gpd.read_file(data_file).set_crs("EPSG:3067", allow_override=True) for data_file in trace_data]
areas = [gpd.read_file(data_file).set_crs("EPSG:3067", allow_override=True) for data_file in area_data]

# Each trace dataset must have a target area
assert len(traces) == len(areas)

In [ ]:
# Assign snap_threshold
snap_threshold = 0.001

# Generate branches and nodes for each trace dataset
branches, nodes = zip(
    *[branches_and_nodes.branches_and_nodes(trace, area, snap_threshold) for trace, area in zip(traces, areas)]
)

In [ ]:
analysis_name = "KB11_sample"
results_folder = "../dev/"

In [ ]:
# All datasets must have names
names = ["KB11"]

assert len(names) == len(traces) == len(areas) == len(branches) == len(nodes)

# Each group must have defined cut offs for traces and branches
groups = ["KB"]

cut_offs_traces = [1.8]
cut_offs_branches = [1.6]

# Assign group to each dataset
datasets_grouped = ["KB"]

assert len(groups) == len(cut_offs_traces) == len(cut_offs_branches)
assert len(names) == len(datasets_grouped)

In [ ]:
set_names = ["1", "2", "3"]
set_limits = [
    (0, 30),
    (30, 90),
    (90, 180),
]

length_set_names = ["0-5", "5-10", "15-10000"]
length_set_limits = [
    (0, 5),
    (5, 10),
    (15, 10000),
]

assert len(set_names) == len(set_limits)
assert len(length_set_names) == len(length_set_limits)

In [ ]:
# Choose which analyses to perform.
choose_your_analyses = {
    "Branches": True,
    "LengthDistributions": True,
    "Azimuths": True,
    "XYI": True,
    "BranchClassification": True,
    "Topology": True,
    "Cross-cuttingAbutting": True,
    "Anisotropy": True,
    "Hexbin": True,
}

In [ ]:
# Analyze trace, branch and node data
main.analyze_datasets(
    traces,
    areas,
    branches,
    nodes,
    names,
    groups,
    cut_offs_traces,
    cut_offs_branches,
    datasets_grouped,
    set_names,
    set_limits,
    length_set_names,
    length_set_limits,
    analysis_name,
    results_folder,
    choose_your_analyses,
)

In [ ]:
# Whether to run contour grid sampling or not
# WARNING: Low cell_width with areally large datasets will have very long processing times.
# Be careful when running with multiscale datasets.
run_contour_sampling = False

In [ ]:
if run_contour_sampling:
    # Assign cell_width
    cell_width = 2.0

    # Create sampled contour grids
    sampled_grids = [
        contour_grid.run_grid_sampling(
            trace, branch, node, cell_width
        ) for trace, branch, node in zip(traces, branches, nodes)
    ]
    # Save sampled contour grids
    [sampled_grid.to_file(f"{results_folder}plots_{analysis_name}/grid_{name}.gpkg", driver="GPKG") for sampled_grid, name in zip(sampled_grids, names)]